## Connect to your GDrive 
In order to train the network on your data, create a directory named `data/`
in the current working directory (cwd) of this notebook (when on colab and connected to gdrive
this would be the `MyDrive/` directory in your gdrive account) and put audio files in it. 

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
# this set the cwd of the notebook
%cd /gdrive/MyDrive 

### Install `mimikit`

In [ ]:
%pip uninstall torchtext -y
%pip install mimikit[colab]==0.4.1

In [ ]:
# colab crashes if following import is done within mimikit
import pytorch_lightning as pl

Generate From Checkpoint

In [ ]:
import mimikit as mmk

# load a checkpoint
ckpt = mmk.Checkpoint(
    root_dir="./",
    id='84e89798e',
    epoch=1
)

# prompt positions in seconds

loop = mmk.GenerateLoopV2.from_config(
    mmk.GenerateLoopV2.Config(
        output_duration_sec=30.,
        prompts_length_sec=1.,
        prompts_position_sec=(
            1.1, 8.5, 46.3
        ),
        batch_size=3,
        display_waveform=True
    ),
    ckpt.dataset,
    ckpt.network
)
for _ in loop.run():
    continue

----------------------------

<img src="https://ktonal.com/k-circle-bw.png" alt="logo" width="75"/>